## Transfer Learning Model to Predict Roof Type

- Data for training model available [here](https://github.com/loosgagnet/Building-detection-and-roof-type-recognition) under 'Dataset 2'.

- Lots of code borrowed from [here](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

Imports: 

In [15]:
import os
from pathlib import Path
from PIL import Image
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from tqdm import tqdm
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import time
import os
from sklearn.model_selection import train_test_split
import copy

Create PyTorch class to load roof data type.

In [16]:
class RoofLoader(Dataset):

    def __init__(self, data_dir='./data/rooftops'):

        self.f_list = [x for x in Path(data_dir).rglob('*.tif')]

    def __len__(self):
        return len(self.f_list)

    def __getitem__(self, idx):

        pth = self.f_list[idx]
        roof_type = os.path.basename(os.path.dirname(pth))

        image = Image.open(pth)
        x = ToTensor()(image)

        y = 1 if roof_type == 'Flat' else 0

        return x, y

Make data loader objects to train model with.

In [17]:
dataset = RoofLoader('/Users/colinbrust/projects/dirtsat-rooftop-index/data/rooftops')

train, val = torch.utils.data.random_split(dataset, [int(len(dataset)*0.75), int(len(dataset)*0.25)])

dataloaders = {
    'train': DataLoader(train, batch_size=16, shuffle=True),
    'val': DataLoader(val, batch_size=16, shuffle=True)
}

dataset_sizes = {
    'train': int(len(dataset)*0.75),
    'val': int(len(dataset)*0.25)
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Function to train model.

In [18]:

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            i = 0
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    print(loss.item())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()


            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

Load pretrained model, define loss function, and train model.

In [19]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=1)